<a href="https://colab.research.google.com/github/mralamdari/CV-Yolo/blob/main/YOLO_v3_ObjectDetection_TensorFlow2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import PIL
import struct # used to convert native Python data types such as strings and numbers into a string of bytes and vice versa
import colorsys
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from tensorflow.python.saved_model import tag_constants

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d aruchomu/data-for-yolo-v3-kernel
!unzip \*.zip && rm *.zip

In [ ]:
!wget -P model_data https://pjreddie.com/media/files/yolov3.weights

!wget -P model_data https://pjreddie.com/media/files/yolov3-tiny.weights

!wget -P model_data https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

!wget -P model_data https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

# Load The Yolo Model

choose between yolo frameworks between; tf and trt


and yolo types between;
yolov3
yolov3-tiny
yolov4
yolov4-tiny


 and decide if you need custom weights 

In [ ]:
def Load_Yolo_model(yolo_framework, yolo_type, yolo_costom_weights, input_size, input_classes):
    
    physical_devices = tf.config.list_physical_devices('GPU')
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except:
        pass

    if yolo_framework == "tf": # TensorFlow detection framework
        
        if yolo_costom_weights:
            checkpoint = f"./checkpoints/{yolo_type}_custom"
            print(f"Loading custom weights from: {checkpoint}")
            yolo = Create_Yolo(input_size=input_size, classes=input_classes)
            yolo.load_weights(checkpoint)
        else:
            Darknet_weights = f'model_data/{yolo_type}.weights'
            print(f"Loading Darknet_weights from: {Darknet_weights}")
            yolo = Create_Yolo(input_size=input_size, classes=input_classes)
            load_yolo_weights(yolo, Darknet_weights) # use Darknet weights
        
    elif yolo_framework == "trt": # TensorRT detection framework
        saved_model_loaded = tf.saved_model.load(yolo_costom_weights, tags=[tag_constants.SERVING])
        signature_keys = list(saved_model_loaded.signatures.keys())
        yolo = saved_model_loaded.signatures['serving_default']

    return yolo  